## Extended Kalman Filter

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib.pyplot as plt
import matplotlib
plt.style.use('paper')
plt.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times"]})

from myst_nb import glue
from src.symbols import *
import src.symbols as symbols
from src.system_equations import *

from IPython.display import display, Math, Latex, Markdown
from sympy.physics.vector.printing import vpprint, vlatex

from src.parameters import df_parameters
p = df_parameters["symbol"]

# Read configs:
conf_path = os.path.join("../../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmms = global_variables["vmms"]

In [ ]:
from wPCC_pipeline.pipelines.preprocess.nodes import filter, assemble_data, add_thrust
from wPCC_pipeline.pipelines.extended_kalman.nodes import create_extended_kalman
from wPCC_pipeline.pipelines.brix.nodes import initial_parameters
from wPCC_pipeline.pipelines.filter_data_extended_kalman.nodes import extended_kalman_filter,extended_kalman_smoother

It has been shown that it is possible to do Parameter identification on perfect (simulated) data with no noise. Such data from physical experiments does not exist in reality however. The measured data will always contain process noise and measurement noise. In order to mitigate this the data is preprocessed using an Extended Kalman filter (EKF). EKF is extending the Kalman Filter (KF) to work on nonlinear systems such as as the VMM:s.

The nonlinear system (the ship) is expressed with a transition model $f(x,u)$ and process noise $w$. $x$ is current state and $u$ are control inputs (rudder angle and thrust). A VMM is used as the transition model for the model test data EKF.

```{math}
:label: eqnonlinearsystem
\dot{x} = f(x,u) + w
```
The state of the system is observed (measured) with the linear observation model $H$ and measurement noise $v$. 

```{math}
:label: eqobserve
y = Hx + v
```
The used EKF recursive algorithm used is summarized in pseudocode below.



$$\dot{x} = f(x,u) + w $$
$$y = Hx + v $$

```{prf:algorithm} Discrete-time extended Kalman filter
:label: ek-algorithm

**Inputs** Initial values: $x_0$, $P_0$, $C_d$, $R_d$, $Q_d$, $E_d$  

**Output** Estimated states: $\hat{x}$, estimated state covariances $\hat{P}$

1. Initial values:
    1. $\hat{x}[0] = x_0$ 
    2. $\hat{P}[0] = P_0$

2. For $k$ in $n$ measurements

    1. KF gain
        1. $K[k]=\hat{P}[k] C_d^T \left(C_d \hat{P}[k] C_d^T + R_d\right)^{-1}$
        2. $I_{KC} = I_n - K[k] C_d$
        
    2. Update
        1. State corrector
            $\hat{x}[k] = \hat{x}[k] + K[k] (y - C_d \hat{x}[k]) $ 
        2. Covariance corrector
            $\hat{P}[k] = I_{KC} \cdot \hat{P}[k] I_{KC}^T + K[k] R_d K^T $
            
    3. Predict
        1. State predictor
            $\hat{x}[k+1] = \hat{x}[k] + h \cdot \hat{f}(\hat{x}[k], u[k])$
        2. Covariance predictor
            $\hat{P}[k+1] = A_d  \hat{P}[k] A_d^T + E_d Q_d E_d^T $
            

	
```

Where $n$ is number of states, $I_n$ is an $n$ * $n$ identity matrix.

The output from the filter are the estimated states: $\hat{x}$ and estimated state covariances $\hat{P}$. $\hat{x}$ represent the most likely estimates, but the estimates have uncertainty that are expressed in $\hat{P}$.

The state of the system is descrived by the ships position, heading, velocities and yaw velocity: 

```{math}
:label: eqstates
x = [x_0,y_0,\psi,u,v,r]^T
```

The initial state $x_0$ is taken as the mean value of the first five measurements, where the velocities are estimated with numeric differentiation.  




$$ x = [x_0,y_0,\psi,u,v,r]^T $$

$C_d$ selects the measured states ($x_0$, $y_0$, $\psi$): 
```{glue:math} eqcd
:label: eqcd
```

In [ ]:
Cd = sp.symbols('C_d')
eq_Cd = sp.Eq(Cd, h*sp.UnevaluatedExpr(sp.Matrix(
        [
            [1, 0, 0, 0, 0, 0],
            [0, 1, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0],
        ]
    )))

glue('eqcd',eq_Cd)

$E_d$ selects the hidden states ($u$, $v, $r$): 

```{glue:math} eqed
:label: eqed
```
Where $h$ is the discrete time step.

In [ ]:
Ed = sp.symbols('E_d')
eq_Ed = sp.Eq(Ed, h*sp.UnevaluatedExpr(sp.Matrix(
        [
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [1, 0, 0],
            [0, 1, 0],
            [0, 0, 1],
        ],
    )))

glue('eqed',eq_Ed)

$R_d$ describes the covariance matrix of the measurement, $Q_d$ is the covariance matrix of the process model. $P_0$ is the initial state covariance.
Selecting good values for these three matrixes is the tricky part in getting the EKF to work well. The amount of expected measurement noise in the data should be inserted in to $R_d$ and the amount of error generated by the process model (VMM) needs to be estimated in $Q_d$. So the choices for these matrixes very much depend on the beleif in the present data and the present process model.

In [ ]:
ship='wpcc'
id = "22773"
vmm = "vmm_abkowitz"
data = catalog.load(f"{ship}.{id}.data")
ek = catalog.load(f"{ ship }.{ vmm }.ek")
ek_covariance_input = catalog.load(f"params:{ship}.updated.ek_covariance_input")
model = catalog.load(f"{ship}.updated.{vmm}.joined.model")

In [ ]:
ek_covariance_input

In [ ]:
x0 = initial_state(data=data)
covariance_matrixes = guess_covariance_matrixes(ek_covariance_input=ek_covariance_input, data=data)

In [ ]:
data2 = data.copy()
data2.iloc[0].update(x0)
result = model.simulate(data2)
df_sim = result.result

In [ ]:
data_noise = df_sim.copy()
data_noise.drop(columns=['u','v','r','u1d','v1d','r1d'], inplace=True)
np.random.seed(42)
data_noise['x0_epsilon']=np.random.normal(0, scale=5*ek_covariance_input['measurement_error_max']['positions']/3, size=len(data_noise))
data_noise['y0_epsilon']=np.random.normal(0, scale=5*ek_covariance_input['measurement_error_max']['positions']/3, size=len(data_noise))
data_noise['psi_epsilon']=np.random.normal(0, scale=5*np.deg2rad(ek_covariance_input['measurement_error_max']['psi']/3), size=len(data_noise))
data_noise['x0']+=data_noise['x0_epsilon']
data_noise['y0']+=data_noise['y0_epsilon']
data_noise['psi']+=data_noise['psi_epsilon']



In [ ]:
from wPCC_pipeline.pipelines.filter_data_extended_kalman.nodes import guess_covariance_matrixes, extended_kalman_filter, initial_state

In [ ]:

ek_filtered, data_ek_filter, time_steps = extended_kalman_filter(ek=ek, data=data_noise, covariance_matrixes=covariance_matrixes, x0=x0, hydrodynamic_derivatives=model.parameters, input_columns=['delta', 'thrust'])

An example of the EKF at work can be seen in [fig](fig_ekf). The filter is run on simulated data with Gausian noise added, so that the "real" state is known.

```{glue:figure} fig_ekf
:figwidth: 1000px
:name: "fig_ekf"

EKF on simulated data (real) with Gausian noise added (raw).
```

In [ ]:
dataframes = {'raw':data_noise,
              'EKF':data_ek_filter,
              'real':df_sim, 
             }

styles = {
    'raw': {'style':'gray','lw':0.2},
    'real' : {'style':'r--', 'lw':1},
    'EKF' : {'style':'b-','alpha':1.0, 'lw':0.4},

}

fig = plot(dataframes, keys=['psi','r','r1d','delta'], styles=styles)
glue('fig_ekf',fig, display=False)

In [ ]:
df_variance = pd.DataFrame(ek_filtered.variance.T, index= ek_filtered.time, columns=['x0','y0','psi','u','v','r'])

In [ ]:
fig,ax=plt.subplots()
df_variance.plot(ax=ax)
ax.set_ylim(0,0.3*df_variance.max().max());